# General Tips on Prompting

Before we get into some big applications of schema engineering I want to equip you with the tools for success.
This notebook is to share some general advice when using prompts to get the most of your models.

Before you might think of prompt engineering as massaging this wall of text, almost like coding in a notepad. But with schema engineering you can get a lot more out of your prompts with a lot less work.


## Classification

For classification we've found theres generally two methods of modeling.

1. using Enums
2. using Literals

Use an enum in Python when you need a set of named constants that are related and you want to ensure type safety, readability, and prevent invalid values. Enums are helpful for grouping and iterating over these constants.

Use literals when you have a small, unchanging set of values that you don't need to group or iterate over, and when type safety and preventing invalid values is less of a concern. Literals are simpler and more direct for basic, one-off values.


In [1]:
%pip install -Uq instructor anthropic

Note: you may need to restart the kernel to use updated packages.


In [1]:
import instructor

from enum import Enum
from pydantic import BaseModel, Field
from typing_extensions import Literal

from anthropic import AnthropicBedrock

client = instructor.from_anthropic(
    AnthropicBedrock(), 
    mode=instructor.Mode.ANTHROPIC_JSON,
    max_tokens=1024
)

# Tip: Do not use auto() as they cast to 1,2,3,4
class House(Enum):
    Gryffindor = "gryffindor"
    Hufflepuff = "hufflepuff"
    Ravenclaw = "ravenclaw"
    Slytherin = "slytherin"


class Character(BaseModel):
    age: int
    name: str
    house: House

    def say_hello(self):
        print(
            f"Hello, I'm {self.name}, I'm {self.age} years old and I'm from {self.house.value.title()}"
        )


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 17, 'name': 'Harry Potter', 'house': <House.Gryffindor: 'gryffindor'>}

In [2]:
resp.say_hello()

Hello, I'm Harry Potter, I'm 17 years old and I'm from Gryffindor


In [3]:
class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 17, 'name': 'Harry Potter', 'house': 'Gryffindor'}

## Arbitrary properties

Often times there are long properties that you might want to extract from data that we can not specify in advanced. We can get around this by defining an arbitrary key value store like so:


In [8]:
from typing import List


class Property(BaseModel):
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property]


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 38,
 'name': 'Severus Snape',
 'house': 'Slytherin',
 'properties': [{'key': 'blood_status', 'value': 'Half-blood'},
  {'key': 'patronus', 'value': 'Doe'},
  {'key': 'occupation',
   'value': 'Potions Master, Defense Against the Dark Arts teacher'}]}

## Limiting the length of lists

In later chapters we'll talk about how to use validators to assert the length of lists but we can also use prompting tricks to enumerate values. Here we'll define a index to count the properties.

In this following example instead of extraction we're going to work on generation instead.


In [11]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 5",
    )


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "Prof Moody from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 73,
 'name': 'Alastor Moody',
 'house': 'Gryffindor',
 'properties': [{'index': '1',
   'key': 'eye_description',
   'value': 'Electric blue magical eye that can see through anything'},
  {'index': '2', 'key': 'signature_phrase', 'value': 'Constant vigilance!'},
  {'index': '3', 'key': 'former_occupation', 'value': 'Auror'},
  {'index': '4',
   'key': 'known_for',
   'value': 'Being paranoid and highly skilled at defensive magic'},
  {'index': '5',
   'key': 'portrayal',
   'value': 'Portrayed by Brendan Gleeson in the Harry Potter films'}]}

## Defining Multiple Entities

Now that we see a single entity with many properties we can continue to nest them into many users


In [13]:
from typing import Iterable


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

age=17 name='Harry Potter' house='Gryffindor'
age=17 name='Ron Weasley' house='Gryffindor'
age=17 name='Hermione Granger' house='Gryffindor'
age=16 name='Draco Malfoy' house='Slytherin'
age=35 name='Severus Snape' house='Slytherin'


## Defining Relationships

Not only can we define lists of users, but with lists of properties we can also easily define lists of references. It's one of the more interesting things I've learned about prompting.


In [15]:
class Character(BaseModel):
    id: int
    name: str
    friends_array: List[int] = Field(description="Relationships to their friends using the id")


resp = client.chat.completions.create(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=[{"role": "user", "content": "5 kids from Harry Potter"}],
    stream=True,
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

id=1 name='Harry Potter' friends_array=[2, 3, 4, 5]
id=2 name='Ron Weasley' friends_array=[1, 3, 4, 5]
id=3 name='Hermione Granger' friends_array=[1, 2, 4, 5]
id=4 name='Neville Longbottom' friends_array=[1, 2, 3, 5]
id=5 name='Luna Lovegood' friends_array=[1, 2, 3, 4]


With the tools we've discussed, we can find numerous real-world applications in production settings. These include extracting action items from transcripts, generating fake data, filling out forms, and creating objects that correspond to generative UI. These simple tricks will be highly useful.


# Missing Data

The Maybe pattern is a concept in functional programming used for error handling. Instead of raising exceptions or returning None, you can use a Maybe type to encapsulate both the result and potential errors.

This pattern is particularly useful when making LLM calls, as providing language models with an escape hatch can effectively reduce hallucinations.

In [16]:
from typing import Optional

class Character(BaseModel):
    age: int
    name: str

class MaybeCharacter(BaseModel):
    result: Optional[Character] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]

In [18]:
def extract(content: str) -> MaybeCharacter:
    return client.chat.completions.create(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        response_model=MaybeCharacter,
        messages=[
            {"role": "user", "content": f"Extract `{content}`"},
        ],
    )

In [21]:
extract("Harry Potter")

MaybeCharacter(result=Character(age=17, name='Harry Potter'), error=False, message=None)

In [24]:
user = extract("404 Error")
print(user)
if user.error:
    raise ValueError(user.message)

result=None error=True message='404 Error'


ValueError: 404 Error